In [1]:
username = "Alex"
env = "integrate"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\" * env * "\\")
include(pathtorepo * "dynamical-systems\\system.jl")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD2

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
function output(idx_I0, idx_U0, I0, U0, u0)
    println("Индекс I0: $idx_I0; I0: $I0");flush(stdout)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_U0,U0, u0)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output_end_iter(ΛΛ, u0_lc)
    println("Спектр: $(ΛΛ)");flush(stdout)
    println("Последняя точка: $(u0_lc)");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [3]:
t = 500; time_attract = 600; tstep = 0.001
trange = range(0.0, t, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [4]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305;

In [5]:
u0 = [10.870367054955267, 0.6670395183801261, 0.44434050730193664];

In [6]:
len = 250
I0range = range( -0.7, -4.00,  length = len )
U0range = range( 0.4,  0.0, length = len )

0.4:-0.001606425702811245:0.0

In [7]:
Λs = zeros(length(I0range), length(U0range), 3)
u0s = zeros(length(I0range), length(U0range), 3);

In [8]:
map_dim = " $(len)x$(len) "
name = " tau_D article2 for Hopf Fold Hom"
format = ".jld2"
namefile_LSE = "LSE" * map_dim * name * format
namefile_u0s = "u0s" * map_dim * name * format

"u0s 250x250  tau_D article2 for Hopf Fold Hom.jld2"

In [9]:
const I0_start_value = I0range[1]

function init_ds(U0_, u0_lc)
    p = [α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_start_value]
    ds = CoupledODEs(TM, u0_lc, p, diffeq = integ_set)
    return ds
end
function init_ds(U0_, I0_, u0_lc)
    p = [α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    ds = CoupledODEs(TM, u0_lc, p,  diffeq = integ_set)
    return ds
end
function goto_attractor(ds_, time_attract)
    tr,_ = trajectory(ds_, time_attract; Δt = tstep)
    u0_lc = tr[end]
    return u0_lc
end
function spectrum(ds_)
    ΛΛ = lyapunovspectrum(ds_, t)
    return ΛΛ
end
function save_output(idx_U0, ΛΛ, u0_lc)
    Λs[1, idx_U0, :] = ΛΛ
    u0s[1, idx_U0, :] = u0_lc
end
function save_output(index_I0, idx_U0, ΛΛ, u0_lc)
    Λs[index_I0, idx_U0, :] = ΛΛ
    u0s[index_I0, idx_U0, :] = u0_lc
end
function save_tofile()
    jldsave(namefile_LSE; Λs)
    jldsave(namefile_u0s; u0s)
end

save_tofile (generic function with 1 method)

In [10]:
for (idx_U0, U0_) in enumerate(U0range)
    
    if idx_U0 == 1 # Проверка на первую итерацию
        # Если использовать while условие не нужно!
        global u0_lc = u0
    end
    
    #output(idx_U0,U0_, u0_lc)
    
    ds = init_ds(U0_, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
    ds = init_ds(U0_, u0_lc)
    ΛΛ = spectrum(ds) # Вычисление спектра
    
    #output_end_iter(ΛΛ, u0_lc)
    
    save_output(idx_U0, ΛΛ, u0_lc) # Сохранение значений
    
    #separate()
end
save_tofile()

In [11]:
for (idx_U0, U0_) in enumerate(U0range)
    for (idx_I0, I0_) in enumerate(I0range)
        
        if idx_I0 == 1 # Если использовать while условие не нужно!
            continue
        end
        
        
        u0_lc = u0s[idx_I0 - 1, idx_U0, :]
        
        # output(idx_I0, idx_U0, I0_, U0_, u0_lc)
        
        ds = init_ds(U0_, I0_, u0_lc)
        u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
        ds = init_ds(U0_, I0_, u0_lc) 
        ΛΛ = spectrum(ds) # Вычисление спектра
        
        # output_end_iter(ΛΛ, u0_lc)
        
        save_output(idx_I0, idx_U0, ΛΛ, u0_lc)
        
        # separate()
    end
    save_tofile()
end